# Computer Vision Project 1: Automatic Grading of Multiple Choice Tests

### Author: Matei Bejan, group 407

## Preliminaries

### Import libraries

In [1]:
import cv2 as cv
import numpy as np
import glob
import os
import tensorflow.keras as keras

model14 = keras.models.load_model('14_recognizer.h5')
model_digit1 = keras.models.load_model('digit1recognizer.h5')
model_digit2 = keras.models.load_model('digit2recognizer.h5')
    
possible_grades = [1.3, 1.6, 1.9, 2.2, 2.5, 2.8, 3.1, 3.4, 3.7, 4.0, 4.3, 4.6, 4.9, 5.2, 5.5, 
                   5.8, 6.1, 6.4, 6.7, 7.0, 7.3, 7.6, 7.9, 8.2, 8.5, 8.8, 9.1, 9.4, 9.7, 10.0]
    
grade_map = {4.9: 12, 6.7: 10, 5.8: 8, 8.5: 3, 9.1: 2, 3.7: 3, 5.5: 16, 4.6: 8, 7.3: 10, 6.4: 17,
             7.0: 7, 7.6: 8, 6.1: 7, 7.9: 4, 8.8: 8, 5.2: 8, 9.7: 2, 8.2: 2, 4.3: 8, 9.4: 2, 4.0: 5}

alpha2digit = {'A': 1, 'B': 2, 'C': 3, 'D': 4}

reference_scan = cv.imread('1_reference_scanned.png')
reference_rotated = cv.imread('2_reference_rotated.png')
reference_perspective = cv.imread('3_reference_perspective.png')

### Load train data

In [2]:
# base_folder = 'images/'

# images = glob.glob(os.path.join(base_folder, "image_*.jpg"))
# images = sorted(images, key = lambda x: int(x.split('_')[1].split('.')[0]))
# image_truths = glob.glob(os.path.join(base_folder, "image_*.txt"))
# image_truths = sorted(image_truths, key = lambda x: int(x.split('_')[1].split('.')[0]))
# rotations = glob.glob(os.path.join(base_folder, "rotation_*.jpg"))
# rotations = sorted(rotations, key = lambda x: int(x.split('_')[1].split('.')[0]))
# perspectives = glob.glob(os.path.join(base_folder, "perspective_*.jpg"))
# perspectives = sorted(perspectives, key = lambda x: int(x.split('_')[1].split('.')[0]))

### Load test data

In [2]:
options_map = {'F1': ['Fizica', '1'], 'F2': ['Fizica', '2'], 
               'F3': ['Fizica', '3'], 'F4': ['Fizica', '4'], 
               'I1': ['Informatica', '1'], 'I2': ['Informatica', '2'], 
               'I3': ['Informatica', '3'], 'I4': ['Informatica', '4']}

scanned_images = glob.glob("test_data/1.scanned/*.jpg")
scanned_images = sorted(scanned_images, key = lambda x: int(x.split('/')[-1].split('_')[0]))

rotated_images = glob.glob("test_data/2.rotated+perspective/*_rotated_*.jpg")
rotated_images = sorted(rotated_images, key = lambda x: int(x.split('/')[-1].split('_')[0]))

perspective_images = glob.glob("test_data/2.rotated+perspective/*_perspective_*.jpg")
perspective_images = sorted(perspective_images, key = lambda x: int(x.split('/')[-1].split('_')[0]))

nanot_images = glob.glob("test_data/3.no_annotation/*.jpg")
nanot_images = sorted(nanot_images, key = lambda x: int(x.split('/')[-1].split('.')[0]))

hw_images = glob.glob("test_data/4.handwritten/*.jpg")
hw_images = sorted(hw_images, key = lambda x: int(x.split('/')[-1].split('_')[0]))

ground_truths = glob.glob("ground-truth-correct-answers/*.txt")

## Define utility functions

In [3]:
def crop_upper_checkbox(img):
    orig_h, orig_w, _ = img.shape
    img = img[int(0.442 * orig_h):int(0.5 * orig_h), int(0.83 * orig_w):int(0.9 * orig_w)]
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    box = cv.adaptiveThreshold(gray, 
                               255, 
                               cv.ADAPTIVE_THRESH_GAUSSIAN_C, 
                               cv.THRESH_BINARY_INV, 
                               101, 3)
    h,w = box.shape
    box = np.rot90(box, 2)
    i = 0
    for row in box:
        if np.mean(row) > 0:
            i += 1
        else:
            break
    box = np.rot90(box, 2)
    box = box[:h-i,:]
    h,w = box.shape
    h1 = 0
    for row in box:
        if np.mean(row) < 50:
            h1 += 1
        else:
            break
    box = np.rot90(box)
    w2 = 0
    for row in box:
        if np.mean(row) < 50:
            w2 += 1
        else:
            break
    box = np.rot90(box)
    h2 = 0
    for row in box:
        if np.mean(row) < 50:
            h2 += 1
        else:
            break
    box = np.rot90(box)
    w1 = 0
    for row in box:
        if np.mean(row) < 50:
            w1 += 1
        else:
            break
    box = np.rot90(box)
    box = box[h1:h-h2,w1:w-w2]
    h, w = box.shape
    h1 = 0
    for row in box:
        if np.mean(row) > 100:
            h1 += 1
        else:
            break
    box = np.rot90(box)
    w2 = 0
    for row in box:
        if np.mean(row) > 100:
            w2 += 1
        else:
            break
    box = np.rot90(box)
    h2 = 0
    for row in box:
        if np.mean(row) > 100:
            h2 += 1
        else:
            break
    box = np.rot90(box)
    w1 = 0
    for row in box:
        if np.mean(row) > 100:
            w1 += 1
        else:
            break
    box = np.rot90(box)
    box = box[h1:h-h2,w1:w-w2]
    return box.copy()

def crop_lower_checkbox(img):
    orig_h, orig_w, _ = img.shape
    img = img[int(0.48 * orig_h):int(0.54 * orig_h), int(0.83 * orig_w):int(0.9 * orig_w)].copy()
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    box =  cv.adaptiveThreshold(gray, 
                                255, 
                                cv.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                cv.THRESH_BINARY_INV, 
                                101, 3)
    h,w = box.shape
    i = 0
    for row in box:
        if np.mean(row) > 0:
            i += 1
        else:
            break
    box = box[i:,:]
    box = np.rot90(box, 2)
    h, w = box.shape
    i = 0
    for row in box:
        if np.mean(row) > 0:
            i += 1
        else:
            break
    box = np.rot90(box, 2)
    box = box[:h-i,:]
    h,w = box.shape
    h1 = 0
    for row in box:
        if np.mean(row) < 50:
            h1 += 1
        else:
            break
    box = np.rot90(box)
    w2 = 0
    for row in box:
        if np.mean(row) < 50:
            w2 += 1
        else:
            break
    box = np.rot90(box)
    h2 = 0
    for row in box:
        if np.mean(row) < 50:
            h2 += 1
        else:
            break
    box = np.rot90(box)
    w1 = 0
    for row in box:
        if np.mean(row) < 50:
            w1 += 1
        else:
            break
    box = np.rot90(box)
    box = box[h1:h-h2,w1:w-w2]
    h, w = box.shape
    h1 = 0
    for row in box:
        if np.mean(row) > 100:
            h1 += 1
        else:
            break
    box = np.rot90(box)
    w2 = 0
    for row in box:
        if np.mean(row) > 100:
            w2 += 1
        else:
            break
    box = np.rot90(box)
    h2 = 0
    for row in box:
        if np.mean(row) > 100:
            h2 += 1
        else:
            break
    box = np.rot90(box)
    w1 = 0
    for row in box:
        if np.mean(row) > 100:
            w1 += 1
        else:
            break
    box = np.rot90(box)
    box = box[h1:h-h2,w1:w-w2]
    return box.copy()

def crop_left_table(img):
    orig_h, orig_w, _ = img.shape
    return img[int(0.525 * orig_h):int(0.87 * orig_h), int(0.2 * orig_w):int(0.38 * orig_w)].copy()

def crop_right_table(img):
    orig_h, orig_w, _ = img.shape
    return img[int(0.525 * orig_h):int(0.87 * orig_h), int(0.73 * orig_w):int(0.9 * orig_w)].copy()

In [4]:
def get_vertical_lines(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    binary = cv.adaptiveThreshold(gray, 
                                  255, 
                                  cv.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                  cv.THRESH_BINARY_INV, 
                                  301, 25)

    lines = cv.HoughLinesP(binary,
                           cv.HOUGH_PROBABILISTIC, 
                           np.pi / 180, 
                           threshold = 150, 
                           minLineLength = 150,
                           maxLineGap = 10)
    
    for line in lines:
        if line[0][0] != line[0][2] and (line[0][0] >= line[0][2] - 25 and line[0][0] <= line[0][2] + 25):
            line[0][2] = line[0][0]
    
    vertical_all = []
    
    for line in lines:
        if line[0][0] == line[0][2]:
            vertical_all.append(line.tolist()[0])
            
    vertical_lines = []

    for line in vertical_all:
        if line not in vertical_all:
            vertical_lines.append(line)
        else:
            flag = False
            for line2 in vertical_lines:
                if not flag and line[0] - 50 <= line2[0] and line2[0] <= line[0] + 50:
                     flag = True
            if not flag:
                vertical_lines.append([line[0], 0, line[0], image.shape[0]])
                
    lines = sorted(vertical_lines, key=lambda line: line[0])
        
    return lines[:5]

def get_horizontal_lines(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    binary = cv.adaptiveThreshold(gray, 
                                  255, 
                                  cv.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                  cv.THRESH_BINARY_INV, 
                                  301, 25)

    lines = cv.HoughLinesP(binary,
                           cv.HOUGH_PROBABILISTIC, 
                           np.pi / 180, 
                           threshold = 150, 
                           minLineLength = 150,
                           maxLineGap = 50)
    
    for line in lines:
        if line[0][1] != line[0][3] and (line[0][1] >= line[0][3] - 25 and line[0][1] <= line[0][3] + 25):
            line[0][3] = line[0][1]

    horizontal_all = []

    for line in lines:
        if line[0][1] == line[0][3] and line[0][1] > 10:
            horizontal_all.append(line.tolist()[0])
            
    horizontal_lines = []

    for line in horizontal_all:
        if line not in horizontal_all:
            horizontal_lines.append(line)
        else:
            flag = False
            for line2 in horizontal_lines:
                if flag == False and line[1] - 50 <= line2[1] and line2[1] <= line[1] + 50:
                     flag = True
            if flag == False:
                horizontal_lines.append([0, line[1], image.shape[1], line[3]])
           
    lines = sorted(horizontal_lines, key=lambda line: line[1], reverse = True)
    
    lines = lines[:16]
    
    lines.reverse()

    return lines

In [5]:
def find_x_from_images(input_image):
    vertical_lines = get_vertical_lines(input_image)
    horizontal_lines = get_horizontal_lines(input_image)
    
    grayscale_image = cv.cvtColor(input_image, cv.COLOR_BGR2GRAY)
    
    image = np.dstack((grayscale_image, grayscale_image, grayscale_image))
    x_color = (0, 255, 0) 
    blank_color = (0, 0, 255)
    
    # No ticks? Multiple ticks on the same row? Check the list's length!
    answer = {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 
              9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: []}
            
    patch_means = []
    
    for i in range(len(horizontal_lines) - 1):
        for j in range(len(vertical_lines) - 1):
            maxx = horizontal_lines[i + 1][1] - horizontal_lines[i][1]
            maxy = vertical_lines[j + 1][0] - vertical_lines[j][0]
            x_min = horizontal_lines[i][1] + maxx // 3
            x_max = horizontal_lines[i + 1][1] - maxx // 4
            y_min = vertical_lines[j][0] + maxy // 4
            y_max = vertical_lines[j + 1][0] - maxy // 4
    
            patch = grayscale_image[x_min:x_max, y_min:y_max].copy()
            
            patch = cv.adaptiveThreshold(patch, 
                                         255, 
                                         cv.ADAPTIVE_THRESH_MEAN_C, 
                                         cv.THRESH_BINARY, 
                                         21, 20)
            
            patch_means.append(np.round(patch.mean()))

    threshold = np.round(np.array(patch_means).mean())

    for i in range(len(horizontal_lines) - 1):
        for j in range(len(vertical_lines) - 1):
            maxx = horizontal_lines[i + 1][1] - horizontal_lines[i][1]
            maxy = vertical_lines[j + 1][0] - vertical_lines[j][0]
            x_min = horizontal_lines[i][1] + maxx // 3
            x_max = horizontal_lines[i + 1][1] - maxx // 4
            y_min = vertical_lines[j][0] + maxy // 4
            y_max = vertical_lines[j + 1][0] - maxy // 4
                
            patch = grayscale_image[x_min:x_max, y_min:y_max].copy()
            
            patch = cv.adaptiveThreshold(patch, 
                                         255, 
                                         cv.ADAPTIVE_THRESH_MEAN_C, 
                                         cv.THRESH_BINARY, 
                                         21, 20)
            
            mean_patch_value = np.round(patch.mean())
            
            if mean_patch_value <= threshold:
                color = x_color
                answer[i + 1].append(j + 1)
            else:
                color = blank_color
    
    return answer

In [6]:
def align_images(image_to_translate, 
                 image_reference, 
                 max_features = 10000,
                 good_match_percent = .1):

    img1_gray = cv.cvtColor(image_to_translate, cv.COLOR_BGR2GRAY)
    img2_gray = cv.cvtColor(image_reference, cv.COLOR_BGR2GRAY)

    orb = cv.ORB_create(max_features)
    keypoints1, descriptors1 = orb.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(img2_gray, None)
    
    matcher = cv.DescriptorMatcher_create(cv.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2, None)

    matches.sort(key=lambda x: x.distance, reverse=False)

    nr_good_matches = int(len(matches) * good_match_percent)
    matches = matches[:nr_good_matches]

    img_matches = cv.drawMatches(image_to_translate, 
                                 keypoints1, 
                                 image_reference, 
                                 keypoints2, 
                                 matches, None)

    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    h, mask = cv.findHomography(points1, points2, cv.RANSAC)

    height, width, channels = image_reference.shape
    img_reg = cv.warpPerspective(image_to_translate, h, (width, height))

    return img_reg

In [7]:
def get_option(image):
    upper = crop_upper_checkbox(image)
    lower = crop_lower_checkbox(image)
        
    if np.mean(upper) > np.mean(lower):
        upper = cv.resize(upper, (28, 28))
        return ('Informatica', np.argmax(model14.predict(upper.reshape(1, 28, 28, 1))))
    elif np.mean(upper) < np.mean(lower):
        lower = cv.resize(lower, (28, 28))
        return ('Fizica', np.argmax(model14.predict(lower.reshape(1, 28, 28, 1))))

## Implementation

### 1. Real world

In [8]:
task1_output = open('output/matei_bejan_407_task1.txt', 'a')

for image_path in scanned_images:
    image = cv.imread(image_path)
    right_table = crop_right_table(image)
    left_table = crop_left_table(image)
    
    answers_left = find_x_from_images(left_table)
    answers_right = find_x_from_images(right_table)
    
    option = options_map[image_path.split('/')[-1].split('_')[-1][:2]]
    
    ground_truth_path = None
    ground_truth = {}
    i = 0
    while ground_truth_path == None:
        if option[0] in ground_truths[i].split('/')[-1] \
        and str(option[1]) in ground_truths[i].split('/')[-1]:
            ground_truth_path = ground_truths[i]
        i += 1

    file1 = open(ground_truth_path, 'r') 
    Lines = file1.readlines() 
    for line in Lines: 
        line = line.split(' ')
        if line[0].isnumeric() and line[0] not in ground_truth:
            ground_truth[int(line[0])] = [alpha2digit[line[1][0]]]
                
    count_correct = 0

    for i in range(1, 16):
        if len(answers_left[i]) == 1 and answers_left[i][0] == ground_truth[i][0]:
            count_correct += 1

    for i1, i2 in list(zip(range(1, 16), range(16, 31))):
        if len(answers_right[i1]) == 1 and answers_right[i1][0] == ground_truth[i2][0]:
            count_correct += 1

    grade = 0.3 * count_correct + 1
    sgr = str(grade)
    if len(sgr) > 3 and sgr[3] == '9':
        grade = float(sgr[:2] + str(int(sgr[2]) + 1))
        
    task1_output.write(image_path.split('/')[-1] + ' ' + str(grade) + '\n')
    
task1_output.close()

### 2. Intermediate

In [13]:
task2_output = open('output/matei_bejan_407_task2.txt', 'a')

for image_path in rotated_images:
    image = cv.imread(image_path)
    aligned = align_images(align_images(image, reference_rotated), reference_scan)
    
    right_table = crop_right_table(aligned)
    left_table = crop_left_table(aligned)
    
    answers_left = find_x_from_images(left_table)
    answers_right = find_x_from_images(right_table)
    
    option = options_map[image_path.split('/')[-1].split('_')[-1][:2]]
    ground_truth_path = None
    ground_truth = {}
    i = 0
    while ground_truth_path == None:
        if option[0] in ground_truths[i].split('/')[-1] \
        and str(option[1]) in ground_truths[i].split('/')[-1]:
            ground_truth_path = ground_truths[i]
        i += 1

    file1 = open(ground_truth_path, 'r') 
    Lines = file1.readlines() 
    for line in Lines: 
        line = line.split(' ')
        if line[0].isnumeric() and line[0] not in ground_truth:
            ground_truth[int(line[0])] = [alpha2digit[line[1][0]]]
                
    count_correct = 0

    for i in range(1, 16):
        if len(answers_left[i]) == 1 and answers_left[i][0] == ground_truth[i][0]:
            count_correct += 1

    for i1, i2 in list(zip(range(1, 16), range(16, 31))):
        if len(answers_right[i1]) == 1 and answers_right[i1][0] == ground_truth[i2][0]:
            count_correct += 1

    grade = 0.3 * count_correct + 1
    sgr = str(grade)
    if len(sgr) > 3 and sgr[3] == '9':
        grade = float(sgr[:2] + str(int(sgr[2]) + 1))

    task2_output.write(image_path.split('/')[-1] + ' ' + str(grade) + '\n')
    
task2_output.close()

In [14]:
task2_output = open('output/matei_bejan_407_task2.txt', 'a')

for image_path in perspective_images:
    image = cv.imread(image_path)
    aligned = align_images(align_images(align_images(image, 
                                                     reference_perspective, 10000, 0.15), 
                                        reference_rotated), 
                           reference_scan)
    
    right_table = crop_right_table(aligned)
    left_table = crop_left_table(aligned)
    
    answers_left = find_x_from_images(left_table)
    answers_right = find_x_from_images(right_table)
    
    option = options_map[image_path.split('/')[-1].split('_')[-1][:2]]
    
    ground_truth_path = None
    ground_truth = {}
    i = 0
    while ground_truth_path == None:
        if option[0] in ground_truths[i].split('/')[-1] \
        and str(option[1]) in ground_truths[i].split('/')[-1]:
            ground_truth_path = ground_truths[i]
        i += 1

    file1 = open(ground_truth_path, 'r') 
    Lines = file1.readlines() 
    for line in Lines: 
        line = line.split(' ')
        if line[0].isnumeric() and line[0] not in ground_truth:
            ground_truth[int(line[0])] = [alpha2digit[line[1][0]]]
                
    count_correct = 0

    for i in range(1, 16):
        if len(answers_left[i]) == 1 and answers_left[i][0] == ground_truth[i][0]:
            count_correct += 1

    for i1, i2 in list(zip(range(1, 16), range(16, 31))):
        if len(answers_right[i1]) == 1 and answers_right[i1][0] == ground_truth[i2][0]:
            count_correct += 1

    grade = 0.3 * count_correct + 1
    sgr = str(grade)
    if len(sgr) > 3 and sgr[3] == '9':
        grade = float(sgr[:2] + str(int(sgr[2]) + 1))

    task2_output.write(image_path.split('/')[-1] + ' ' + str(grade) + '\n')
    
task2_output.close()

### 3. No annotations

In [11]:
task3_output = open('output/matei_bejan_407_task3.txt', 'a')

for image_path in nanot_images:
    image = cv.imread(image_path)
    if np.mean(image) < 230:
        aligned = align_images(align_images(align_images(image, 
                                                         reference_perspective, 10000, .15), 
                                            reference_rotated), 
                               reference_scan)
    else:
        aligned = image
        
    right_table = crop_right_table(aligned)
    left_table = crop_left_table(aligned)
    
    answers_left = find_x_from_images(left_table)
    answers_right = find_x_from_images(right_table)
    
    option = get_option(aligned)

    if option != None:
        ground_truth_path = None
        ground_truth = {}
        i = 0
        while ground_truth_path == None:
            if option[0] in ground_truths[i].split('/')[-1] \
            and str(option[1]) in ground_truths[i].split('/')[-1]:
                ground_truth_path = ground_truths[i]
            i += 1

        file1 = open(ground_truth_path, 'r') 
        Lines = file1.readlines() 
        for line in Lines: 
            line = line.split(' ')
            if line[0].isnumeric() and line[0] not in ground_truth:
                ground_truth[int(line[0])] = [alpha2digit[line[1][0]]]

        count_correct = 0

        for i in range(1, 16):
            if len(answers_left[i]) == 1 and answers_left[i][0] == ground_truth[i][0]:
                count_correct += 1

        for i1, i2 in list(zip(range(1, 16), range(16, 31))):
            if len(answers_right[i1]) == 1 and answers_right[i1][0] == ground_truth[i2][0]:
                count_correct += 1

        grade = 0.3 * count_correct + 1
        sgr = str(grade)
        if len(sgr) > 3 and sgr[3] == '9':
            grade = float(sgr[:2] + str(int(sgr[2]) + 1))

        task3_output.write(image_path.split('/')[-1] + ' ' + str(grade) + '\n')
    else:
        task3_output.write(image_path.split('/')[-1] + '6.4\n')
        
task3_output.close()

### 4. Handwritten recognition

In [12]:
task4_output = open('output/matei_bejan_407_task4.txt', 'a')

total_correct = 0

for image_path in hw_images:
    image = cv.imread(image_path)
    orig_h, orig_w, _ = image.shape
    image = image[int(0.29 * orig_h):int(0.33 * orig_h), int(0.09 * orig_w):int(0.153 * orig_w)]
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    lower = np.array([170,50,50])
    upper = np.array([180,255,255])
    mask = cv.inRange(image, lower, upper)
    mask = cv.GaussianBlur(mask, (7, 7), 1)
    mask2 = []
    mask = np.rot90(mask)
    for line in mask:
        if np.mean(line) > 0:
            mask2.append(line)
    mask = np.array(mask2)
    mask2 = []
    mask = np.rot90(mask)
    for line in mask:
        if np.mean(line) > 0:
            mask2.append(line)
    mask = np.array(mask2)
    mask2 = []
    mask = np.rot90(mask)
    for line in mask:
        if np.mean(line) > 0:
            mask2.append(line)
    mask = np.array(mask2)
    mask = np.rot90(mask)

    h, w = mask.shape
    fp = mask[:,:w // 3 + 5]
    sp = mask[:,w // 2 - 10:]
    sp = sp[:,:w // 2 + 10]
    fp = cv.resize(fp, (28, 28))
    fp = np.expand_dims(fp, 2)
    sp = cv.resize(sp, (28, 28))
    sp = np.expand_dims(sp, 2)
    
    p1 = np.argmax(model_digit1.predict(fp.reshape(1, 28, 28, 1)))
    p2 = np.argmax(model_digit2.predict(sp.reshape(1, 28, 28, 1)))

    predicted_grade = float(p1 * 10 + p2) / 10
    if predicted_grade <= 1:
        predicted_grade = 6.4
    elif predicted_grade not in possible_grades:
        first_digit = int(str(predicted_grade)[0])
        second_digit = None
        maxi = 0
        for key, val in grade_map.items():
            if int(str(key)[0]) == first_digit and val > maxi:
                maxi = val
                second_digit = int(str(key)[2])
        predicted_grade = float(first_digit * 10 + second_digit) / 10

    task4_output.write(image_path.split('/')[-1] + ' ' + str(predicted_grade) + '\n')
        
task4_output.close()